In [1]:
from torchtext.data.utils import get_tokenizer
from torchtext.vocab import build_vocab_from_iterator
from torchtext.datasets import multi30k, Multi30k
from typing import Iterable, List
import torch
from torchtext.data.utils import get_tokenizer
from datasets import load_dataset
from tqdm import tqdm

In [2]:
SRC_LANGUAGE = "de"
TGT_LANGUAGE = "en"

In [3]:
from torch.utils.data import Dataset, DataLoader
class WMT16Dataset(Dataset):
    def __init__(self, split):
        self.split = split
        self.data = load_dataset('wmt16', SRC_LANGUAGE + "-" + TGT_LANGUAGE, split=self.split)
    def __len__(self):
        return len(self.data)
    def __getitem__(self, index):
        srcText = self.data[index]['translation'][SRC_LANGUAGE]
        tgtText = self.data[index]['translation'][TGT_LANGUAGE]
        return srcText, tgtText

In [ ]:
!python -m spacy download en_core_web_sm
!python -m spacy download de_core_news_sm

In [5]:
TRAIN_SPLIT_SIZE = 30000

In [ ]:
token_transform = {}
vocab_transform = {}
token_transform[SRC_LANGUAGE] = get_tokenizer('spacy', language='de_core_news_sm')
token_transform[TGT_LANGUAGE] = get_tokenizer('spacy', language='en_core_web_sm')
def yield_tokens(dataIter: Iterable, language: str) -> List[str]:
    with tqdm(total=len(dataIter.data), desc="Evaluating") as pbar:
        for dataSample in dataIter.data:
            yield token_transform[language](dataSample["translation"][language])
            pbar.update(1)
UNK_IDX, PAD_IDX, BOS_IDX, EOS_IDX = 0, 1, 2, 3
special_symbols = ['<unk>', '<pad>', '<bos>', '<eos>']
for ln in [SRC_LANGUAGE, TGT_LANGUAGE]:
    train_iter = WMT16Dataset(split=f'train[:{TRAIN_SPLIT_SIZE}]')
    vocab_transform[ln] = build_vocab_from_iterator(yield_tokens(train_iter, ln),
                                                    min_freq=1,
                                                    specials=special_symbols,
                                                    special_first=True)
for ln in [SRC_LANGUAGE, TGT_LANGUAGE]:
    vocab_transform[ln].set_default_index(UNK_IDX)

In [7]:
from torch import Tensor
import torch
import torch.nn as nn
from torch.nn import Transformer
import math

DEVICE = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

In [8]:
# helper Module that adds positional encoding to the token embedding to introduce a notion of word order.
class PositionalEncoding(nn.Module):
    def __init__(self,
                 emb_size: int,
                 dropout: float,
                 maxlen: int = 5000):
        super(PositionalEncoding, self).__init__()
        den = torch.exp(- torch.arange(0, emb_size, 2)* math.log(10000) / emb_size)
        pos = torch.arange(0, maxlen).reshape(maxlen, 1)
        pos_embedding = torch.zeros((maxlen, emb_size))
        pos_embedding[:, 0::2] = torch.sin(pos * den)
        pos_embedding[:, 1::2] = torch.cos(pos * den)
        pos_embedding = pos_embedding.unsqueeze(-2)
        self.dropout = nn.Dropout(dropout)
        self.register_buffer('pos_embedding', pos_embedding)
    def forward(self, token_embedding: Tensor):
        return self.dropout(token_embedding + self.pos_embedding[:token_embedding.size(0), :])

# helper Module to convert tensor of input indices into corresponding tensor of token embeddings
class TokenEmbedding(nn.Module):
    def __init__(self, vocab_size: int, emb_size):
        super(TokenEmbedding, self).__init__()
        self.embedding = nn.Embedding(vocab_size, emb_size)
        self.emb_size = emb_size
    def forward(self, tokens: Tensor):
        return self.embedding(tokens.long()) * math.sqrt(self.emb_size)

# Seq2Seq Network
class Seq2SeqTransformer(nn.Module):
    def __init__(self,
                 num_encoder_layers: int,
                 num_decoder_layers: int,
                 emb_size: int,
                 nhead: int,
                 src_vocab_size: int,
                 tgt_vocab_size: int,
                 dim_feedforward: int = 512,
                 dropout: float = 0.1):
        super(Seq2SeqTransformer, self).__init__()
        self.transformer = Transformer(d_model=emb_size,
                                       nhead=nhead,
                                       num_encoder_layers=num_encoder_layers,
                                       num_decoder_layers=num_decoder_layers,
                                       dim_feedforward=dim_feedforward,
                                       dropout=dropout)
        self.generator = nn.Linear(emb_size, tgt_vocab_size)
        self.src_tok_emb = TokenEmbedding(src_vocab_size, emb_size)
        self.tgt_tok_emb = TokenEmbedding(tgt_vocab_size, emb_size)
        self.positional_encoding = PositionalEncoding(
            emb_size, dropout=dropout)
    def forward(self,
                src: Tensor,
                trg: Tensor,
                src_mask: Tensor,
                tgt_mask: Tensor,
                src_padding_mask: Tensor,
                tgt_padding_mask: Tensor,
                memory_key_padding_mask: Tensor):
        src_emb = self.positional_encoding(self.src_tok_emb(src))
        tgt_emb = self.positional_encoding(self.tgt_tok_emb(trg))
        outs = self.transformer(src_emb, tgt_emb, src_mask, tgt_mask, None,
                                src_padding_mask, tgt_padding_mask, memory_key_padding_mask)
        return self.generator(outs)
    def encode(self, src: Tensor, src_mask: Tensor):
        return self.transformer.encoder(self.positional_encoding(
                            self.src_tok_emb(src)), src_mask)
    def decode(self, tgt: Tensor, memory: Tensor, tgt_mask: Tensor):
        return self.transformer.decoder(self.positional_encoding(
                          self.tgt_tok_emb(tgt)), memory,
                          tgt_mask)

def generate_square_subsequent_mask(sz):
    mask = (torch.triu(torch.ones((sz, sz), device=DEVICE)) == 1).transpose(0, 1)
    mask = mask.float().masked_fill(mask == 0, float('-inf')).masked_fill(mask == 1, float(0.0))
    return mask
def create_mask(src, tgt):
    src_seq_len = src.shape[0]
    tgt_seq_len = tgt.shape[0]
    tgt_mask = generate_square_subsequent_mask(tgt_seq_len)
    src_mask = torch.zeros((src_seq_len, src_seq_len),device=DEVICE).type(torch.bool)
    src_padding_mask = (src == PAD_IDX).transpose(0, 1)
    tgt_padding_mask = (tgt == PAD_IDX).transpose(0, 1)
    return src_mask, tgt_mask, src_padding_mask, tgt_padding_mask

In [9]:
torch.manual_seed(0)

SRC_VOCAB_SIZE = len(vocab_transform[SRC_LANGUAGE])
TGT_VOCAB_SIZE = len(vocab_transform[TGT_LANGUAGE])
EMB_SIZE = 512
NHEAD = 8
FFN_HID_DIM = 512
BATCH_SIZE = 32
NUM_ENCODER_LAYERS = 3
NUM_DECODER_LAYERS = 3

transformer = Seq2SeqTransformer(NUM_ENCODER_LAYERS, NUM_DECODER_LAYERS, EMB_SIZE,
                                 NHEAD, SRC_VOCAB_SIZE, TGT_VOCAB_SIZE, FFN_HID_DIM)

for p in transformer.parameters():
    if p.dim() > 1:
        nn.init.xavier_uniform_(p)

transformer = transformer.to(DEVICE)
loss_fn = torch.nn.CrossEntropyLoss(ignore_index=PAD_IDX)
optimizer = torch.optim.Adam(transformer.parameters(), lr=0.0001, betas=(0.9, 0.98), eps=1e-9)

/opt/conda/lib/python3.10/site-packages/torch/nn/modules/transformer.py:282: UserWarning: enable_nested_tensor is True, but self.use_nested_tensor is False because encoder_layer.self_attn.batch_first was not True(use batch_first for better inference performance)
  warnings.warn(f"enable_nested_tensor is True, but self.use_nested_tensor is False because {why_not_sparsity_fast_path}")


In [10]:
from torch.nn.utils.rnn import pad_sequence

# helper function to club together sequential operations
def sequential_transforms(*transforms):
    def func(txt_input):
        for transform in transforms:
            txt_input = transform(txt_input)
        return txt_input
    return func

# function to add BOS/EOS and create tensor for input sequence indices
def tensor_transform(token_ids: List[int]):
    return torch.cat((torch.tensor([BOS_IDX]),
                      torch.tensor(token_ids),
                      torch.tensor([EOS_IDX])))

# ``src`` and ``tgt`` language text transforms to convert raw strings into tensors indices
text_transform = {}
for ln in [SRC_LANGUAGE, TGT_LANGUAGE]:
    text_transform[ln] = sequential_transforms(token_transform[ln], #Tokenization
                                               vocab_transform[ln], #Numericalization
                                               tensor_transform) # Add BOS/EOS and create tensor


# function to collate data samples into batch tensors
def collate_fn(batch):
    src_batch, tgt_batch = [], []
    for src_sample, tgt_sample in batch:
        src_batch.append(text_transform[SRC_LANGUAGE](src_sample.rstrip("\n")))
        tgt_batch.append(text_transform[TGT_LANGUAGE](tgt_sample.rstrip("\n")))

    src_batch = pad_sequence(src_batch, padding_value=PAD_IDX)
    tgt_batch = pad_sequence(tgt_batch, padding_value=PAD_IDX)
    return src_batch, tgt_batch

In [ ]:
from torch.utils.data import DataLoader

def train_epoch(model, optimizer, trainDataLoader):
    model.train()
    losses = 0
    for src, tgt in trainDataLoader:
        src = src.to(DEVICE)
        tgt = tgt.to(DEVICE)
        tgt_input = tgt[:-1, :]
        src_mask, tgt_mask, src_padding_mask, tgt_padding_mask = create_mask(src, tgt_input)
        logits = model(src, tgt_input, src_mask, tgt_mask,src_padding_mask, tgt_padding_mask, src_padding_mask)
        optimizer.zero_grad()
        tgt_out = tgt[1:, :]
        loss = loss_fn(logits.reshape(-1, logits.shape[-1]), tgt_out.reshape(-1))
        loss.backward()
        optimizer.step()
        losses += loss.item()
    return losses/len(list(trainDataLoader))

def evaluate_loss(model):
    model.eval()
    losses = 0
    val_iter = WMT16Dataset(split="validation")
    val_dataloader = DataLoader(val_iter, batch_size=BATCH_SIZE, collate_fn=collate_fn)
    for src, tgt in val_dataloader:
        src = src.to(DEVICE)
        tgt = tgt.to(DEVICE)
        tgt_input = tgt[:-1, :]
        src_mask, tgt_mask, src_padding_mask, tgt_padding_mask = create_mask(src, tgt_input)
        logits = model(src, tgt_input, src_mask, tgt_mask,src_padding_mask, tgt_padding_mask, src_padding_mask)
        tgt_out = tgt[1:, :]
        loss = loss_fn(logits.reshape(-1, logits.shape[-1]), tgt_out.reshape(-1))
        losses += loss.item()
    return losses / len(list(val_dataloader))

In [27]:
from timeit import default_timer as timer
NUM_CHUNKS = 10
CHUNK_SIZE = 5000
train_losses = []
val_losses = []
NUM_EPOCHS = 5

with tqdm(total=NUM_CHUNKS, desc="Training") as pbar:
    for i in range(NUM_CHUNKS):
        trainIter = WMT16Dataset(split=f"train[{int(i*CHUNK_SIZE)}:{int((i+1)*CHUNK_SIZE)}]")
        trainDataLoader = DataLoader(trainIter, batch_size=BATCH_SIZE, collate_fn=collate_fn)
        for epoch in range(1, NUM_EPOCHS+1):
            start_time = timer()
            train_loss = train_epoch(transformer, optimizer, trainDataLoader)
            end_time = timer()
            val_loss = evaluate_loss(transformer)
            train_losses.append(train_loss)
            val_losses.append(val_loss)
            print(f"Epoch: {epoch} Train loss: {train_loss:.3f}, Val loss: {val_loss:.3f}, ",
                  f"Epoch time = {(end_time - start_time):.3f}s")
        pbar.update(1)

Training:   0%|          | 0/10 [00:00<?, ?it/s]

Epoch: 1 Train loss: 2.936, Val loss: 5.063,  Epoch time = 16.236s
Epoch: 2 Train loss: 2.523, Val loss: 5.123,  Epoch time = 16.246s
Epoch: 3 Train loss: 2.260, Val loss: 5.136,  Epoch time = 16.270s
Epoch: 4 Train loss: 2.040, Val loss: 5.223,  Epoch time = 16.263s


Training:  10%|█         | 1/10 [01:42<15:20, 102.29s/it]

Epoch: 5 Train loss: 1.844, Val loss: 5.428,  Epoch time = 16.236s
Epoch: 1 Train loss: 2.869, Val loss: 5.349,  Epoch time = 16.542s
Epoch: 2 Train loss: 2.455, Val loss: 5.463,  Epoch time = 16.481s
Epoch: 3 Train loss: 2.191, Val loss: 5.601,  Epoch time = 16.664s
Epoch: 4 Train loss: 1.975, Val loss: 5.723,  Epoch time = 16.644s


Training:  20%|██        | 2/10 [03:25<13:43, 102.88s/it]

...

Epoch: 5 Train loss: 1.564, Val loss: 5.016,  Epoch time = 15.502s
Epoch: 1 Train loss: 2.567, Val loss: 4.584,  Epoch time = 15.673s
Epoch: 2 Train loss: 2.134, Val loss: 4.699,  Epoch time = 15.891s
Epoch: 3 Train loss: 1.868, Val loss: 4.797,  Epoch time = 15.869s
Epoch: 4 Train loss: 1.651, Val loss: 4.906,  Epoch time = 15.892s


Training:  90%|█████████ | 9/10 [15:15<01:40, 100.54s/it]

Epoch: 5 Train loss: 1.483, Val loss: 5.040,  Epoch time = 15.852s
Epoch: 1 Train loss: 2.633, Val loss: 4.536,  Epoch time = 15.654s
Epoch: 2 Train loss: 2.199, Val loss: 4.634,  Epoch time = 15.653s
Epoch: 3 Train loss: 1.927, Val loss: 4.803,  Epoch time = 15.698s
Epoch: 4 Train loss: 1.714, Val loss: 4.905,  Epoch time = 15.654s


Training: 100%|██████████| 10/10 [16:54<00:00, 101.45s/it]

Epoch: 5 Train loss: 1.537, Val loss: 5.021,  Epoch time = 15.692s


In [28]:
torch.save(transformer, '/kaggle/working/transformer.pt')
torch.save(optimizer, '/kaggle/working/optimizer.pt')

In [31]:
from IPython.display import FileLink
FileLink(r"transformer.pt")

/kaggle/working/transformer.pt

In [36]:
FileLink(r"optimizer.pt")

/kaggle/working/optimizer.pt

In [32]:
print(train_losses)
print(val_losses)

[2.93555835098218, 2.5228596956107268, 2.260005887146968, 2.0395615283091355, 1.8436836323161034, 2.8686448783631535, 2.4549124058644485, 2.190834797112046, 1.9751192308535241, 1.7891666031187508, 2.8045938379445654, 2.3813386973302078, 2.1140699918103065, 1.9028279530774257, 1.7178909634328952, 2.8743009225578064, 2.4328575081126704, 2.150474805361146, 1.9308944374892363, 1.7402013593418584, 2.726984756008075, 2.306718485370563, 2.0353990535067905, 1.8195179761595028, 1.6423388916975374, 2.701938828085638, 2.2619586028870504, 1.9846835751442393, 1.763874110142896, 1.5800462896656837, 2.6733564609175273, 2.2482146776405867, 1.9840572085350183, 1.7747709781500944, 1.5923832203172574, 2.649979968739163, 2.226173712949085, 1.9562417686365212, 1.7434210879787517, 1.564313498272258, 2.5668379455615002, 2.1337405321704352, 1.8676150475338007, 1.6506046098508653, 1.48323946659732, 2.6328592642097717, 2.1990925096402503, 1.9273807121689912, 1.7138988125096462, 1.537241074309987]
[5.06340048243

In [ ]:
# function to generate output sequence using greedy algorithm
def greedy_decode(model, src, src_mask, max_len, start_symbol):
    src = src.to(DEVICE)
    src_mask = src_mask.to(DEVICE)
    memory = model.encode(src, src_mask)
    ys = torch.ones(1, 1).fill_(start_symbol).type(torch.long).to(DEVICE)
    for i in range(max_len-1):
        memory = memory.to(DEVICE)
        tgt_mask = (generate_square_subsequent_mask(ys.size(0))
                    .type(torch.bool)).to(DEVICE)
        out = model.decode(ys, memory, tgt_mask)
        out = out.transpose(0, 1)
        prob = model.generator(out[:, -1])
        _, next_word = torch.max(prob, dim=1)
        next_word = next_word.item()
        ys = torch.cat([ys,
                        torch.ones(1, 1).type_as(src.data).fill_(next_word)], dim=0)
        if next_word == EOS_IDX:
            break
    return ys

# actual function to translate input sentence into target language
def translate(model: torch.nn.Module, src_sentence: str):
    model.eval()
    src = text_transform[SRC_LANGUAGE](src_sentence).view(-1, 1)
    num_tokens = src.shape[0]
    src_mask = (torch.zeros(num_tokens, num_tokens)).type(torch.bool)
    tgt_tokens = greedy_decode(
        model,  src, src_mask, max_len=num_tokens + 5, start_symbol=BOS_IDX).flatten()
    return " ".join(vocab_transform[TGT_LANGUAGE].lookup_tokens(list(tgt_tokens.cpu().numpy()))).replace("<bos>", "").replace("<eos>", "")

In [ ]:
!pip install evaluate
!pip install -U nltk
!pip install bert_score

In [34]:
import evaluate
from tqdm import tqdm
from pprint import pprint

In [35]:
translatedSamples, tgtSamples = [], []
dataIter = WMT16Dataset(split="validation")
with tqdm(total=len(dataIter), desc="Evaluating") as pbar:
    for dataSample in dataIter:
        translatedSamples.append(translate(transformer, dataSample[0]))
        tgtSamples.append(dataSample[1])
        pbar.update(1)

Evaluating: 100%|██████████| 2169/2169 [03:36<00:00, 10.03it/s]


In [37]:
for maxOrder in range(1, 5):
    print(f"\nBLEU{maxOrder}")
    print(evaluate.load("bleu").compute(predictions=translatedSamples, references=tgtSamples, 
                                        max_order=maxOrder))


BLEU1
{'bleu': 0.26750774095790236, 'precisions': [0.26750774095790236], 'brevity_penalty': 1.0, 'length_ratio': 1.3073229550201322, 'translation_length': 60716, 'reference_length': 46443}

BLEU2
{'bleu': 0.1368197636084783, 'precisions': [0.26750774095790236, 0.06997796642014108], 'brevity_penalty': 1.0, 'length_ratio': 1.3073229550201322, 'translation_length': 60716, 'reference_length': 46443}

BLEU3
{'bleu': 0.07506759081825795, 'precisions': [0.26750774095790236, 0.06997796642014108, 0.02259746709709461], 'brevity_penalty': 1.0, 'length_ratio': 1.3073229550201322, 'translation_length': 60716, 'reference_length': 46443}

BLEU4
{'bleu': 0.04331273866531156, 'precisions': [0.26750774095790236, 0.06997796642014108, 0.02259746709709461, 0.008319651718349351], 'brevity_penalty': 1.0, 'length_ratio': 1.3073229550201322, 'translation_length': 60716, 'reference_length': 46443}


In [38]:
print(f"METEOR: {evaluate.load('meteor').compute(predictions=translatedSamples, references=tgtSamples)}")

[nltk_data] Downloading package wordnet to /usr/share/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package punkt to /usr/share/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package omw-1.4 to /usr/share/nltk_data...
[nltk_data]   Package omw-1.4 is already up-to-date!


METEOR: {'meteor': 0.25484074117445193}


In [39]:
bertScores = evaluate.load("bertscore").compute(predictions=translatedSamples,
                                                references=tgtSamples, lang="de")
bertPrecisions = bertScores['precision']
bertRecalls = bertScores['recall']
bertF1s = bertScores['f1']
print(f"BERT avg precision: {(sum(bertPrecisions)/len(bertPrecisions))}")
print(f"BERT avg recall: {(sum(bertRecalls)/len(bertRecalls))}")
print(f"BERT avg f1: {(sum(bertF1s)/len(bertF1s))}")

BERT avg precision: 0.689967467687052
BERT avg recall: 0.7084532111622102
BERT avg f1: 0.6987526666573963


In [40]:
translatedSamples, tgtSamples = [], []
dataIter = WMT16Dataset(split="test")
with tqdm(total=len(dataIter), desc="Evaluating") as pbar:
    for dataSample in dataIter:
        translatedSamples.append(translate(transformer, dataSample[0]))
        tgtSamples.append(dataSample[1])
        pbar.update(1)

Evaluating: 100%|██████████| 2999/2999 [05:04<00:00,  9.84it/s]


In [41]:
for maxOrder in range(1, 5):
    print(f"\nBLEU{maxOrder}")
    print(evaluate.load("bleu").compute(predictions=translatedSamples, references=tgtSamples, 
                                        max_order=maxOrder))


BLEU1
{'bleu': 0.26845394966998626, 'precisions': [0.26845394966998626], 'brevity_penalty': 1.0, 'length_ratio': 1.3374194856438808, 'translation_length': 85754, 'reference_length': 64119}

BLEU2
{'bleu': 0.14011584077381548, 'precisions': [0.26845394966998626, 0.07313153283789499], 'brevity_penalty': 1.0, 'length_ratio': 1.3374194856438808, 'translation_length': 85754, 'reference_length': 64119}

BLEU3
{'bleu': 0.07892169272467453, 'precisions': [0.26845394966998626, 0.07313153283789499, 0.025038868549074677], 'brevity_penalty': 1.0, 'length_ratio': 1.3374194856438808, 'translation_length': 85754, 'reference_length': 64119}

BLEU4
{'bleu': 0.046641883104234394, 'precisions': [0.26845394966998626, 0.07313153283789499, 0.025038868549074677, 0.009627535533292512], 'brevity_penalty': 1.0, 'length_ratio': 1.3374194856438808, 'translation_length': 85754, 'reference_length': 64119}


In [42]:
print(f"METEOR: {evaluate.load('meteor').compute(predictions=translatedSamples, references=tgtSamples)}")

[nltk_data] Downloading package wordnet to /usr/share/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package punkt to /usr/share/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package omw-1.4 to /usr/share/nltk_data...
[nltk_data]   Package omw-1.4 is already up-to-date!


METEOR: {'meteor': 0.26291187738117555}


In [43]:
bertScores = evaluate.load("bertscore").compute(predictions=translatedSamples,
                                                references=tgtSamples, lang="de")
bertPrecisions = bertScores['precision']
bertRecalls = bertScores['recall']
bertF1s = bertScores['f1']
print(f"BERT avg precision: {(sum(bertPrecisions)/len(bertPrecisions))}")
print(f"BERT avg recall: {(sum(bertRecalls)/len(bertRecalls))}")
print(f"BERT avg f1: {(sum(bertF1s)/len(bertF1s))}")

BERT avg precision: 0.6901324533867653
BERT avg recall: 0.7094622124390191
BERT avg f1: 0.6993435171891467
